## Load Libraries and Data

In [168]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [169]:
# Load stocks dataframe

df_stocks = pd.read_csv("stock_prices_8.csv")

FileNotFoundError: [Errno 2] File b'stock_prices_8.csv' does not exist: b'stock_prices_8.csv'

In [ ]:
# Create stock price relatives

def compute_price_relatives(df):
    
    df = df.copy(deep=True)

    for stock in list(df.columns)[1:]:
        df[stock] = df[stock]/df[stock].shift(+1)
    
    df = df.iloc[1:, :]
    
    return df

In [ ]:
def plot_stock_performance(df_stocks, stocks, title_text, yaxis_text):

    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=df_stocks["Date"],
                             y=df_stocks[stocks[0]],
                             mode='lines',
                             name=stocks[0]))
    
    fig.add_trace(go.Scatter(x=df_stocks["Date"],
                             y=df_stocks[stocks[1]],
                             mode='lines',
                             name=stocks[1]))
    
    fig.update_layout(title=title_text, yaxis_title=yaxis_text)
    
    return fig

### Constantly Rebalanced Portfolio in Hindsight

In [ ]:
# What is the optimal constantly rebalanced portfolio allocation b = [b1, 1-b1], such that the compounded growth is maximized?

In [ ]:
def cumulative_wealth_multiple(df_stocks_relatives, b_1, stocks):
    
    # What is the optimal constantly rebalanced portfolio allocation b = [b1, 1-b1], 
    # such that the compounded growth is maximized?
    
    """
    b1: Amount of stock 1 (as ranked in stocks) in the portfolio
    """

    b_2 = 1 - b_1
    b = np.array([b_1, b_2])
    
    x_1 = np.array(df_stocks_relatives[stocks[0]])
    x_2 = np.array(df_stocks_relatives[stocks[1]])
    
    x_matrix = np.vstack((x_1, x_2))
    period_returns = np.array(np.matmul(b, x_matrix))
    
    return np.prod(period_returns)

In [ ]:
def hindsight_constantly_rebalanced_portfolio(df_stocks_relatives, stocks):

    frac_stock_1 = np.arange(0, 1.01, 0.01)
    wealth_multiple_hindsight = np.array([cumulative_wealth_multiple(df_stocks_relatives=df_stocks_relatives, 
                                                                     b_1=b_stock_1, stocks=stocks) for b_stock_1 in frac_stock_1])

    df_hindsight = pd.DataFrame(data={"fraction": frac_stock_1, 
                                      "wealth_multiple": wealth_multiple_hindsight})
    
    optimal_alloc_1 = df_hindsight.iloc[df_hindsight['wealth_multiple'].idxmax(), 0]
    best_hindsight_multiplier = df_hindsight['wealth_multiple'].max()
    
    return df_hindsight, optimal_alloc_1, best_hindsight_multiplier

In [ ]:
def plot_optimal_hindsight_rebalanced_portfolio(df_hindsight, stocks):

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df_hindsight["fraction"],
                             y=df_hindsight["wealth_multiple"],
                             mode='markers+lines',
                             name=stocks[0]))
    
    fig.update_layout(title="Wealth Multiplier of Constantly Rebalanced Portfolio in Hindsight", 
                      yaxis_title="Wealth Multiple at the end of period",
                      xaxis_title= "Fraction of Portfolio in " + stocks[0])
    
    return fig

## Tom Cover Universal Portfolio Strategy

In [ ]:
def S_k(b_k, x_k):
    
    """
    b_k = [b_1_k, b_2_k] portfolio allocation at time step k
    x_k = [x_1_k, x_2_k]^k wealth multiplier at time step k
    """
    
    return np.prod(np.matmul(b_k, x_k))

In [ ]:
def S_n(b_n, x_n, n):
    
    """
    Computes the final wealth factor after n steps
    
    b_n = list containing arrays of b_k
    x_k = observed wealth multiplier of sequences
    """
    product_term = []
    
    for k in range(1, n+1):
        b_k = b_n[k-1]
        x_k = x_n[:, k-1]
        product_term.append(np.matmul(b_k, x_k))
        
    result = np.prod(np.array(product_term))
    
    return result

In [ ]:
def b_next(x_n, k, number_of_splits=20):
    
    """
    Rebalanced portfolio of securities at each time steps (only for m = 2)
    
    The integral over interval [0,1] is split into "number_of_splits" chunks
    
    x_n = matrix of relative performance (m x n : m stocks and n days)
    k = index of current timestep
    number_of_splits = controls the resolution of the integration
    """
    
    # Calculate Numerator and Denominator 
    
    numerator_terms = []
    denominator_terms = []
    
    for i in range(0, number_of_splits+1):
        b_i = np.array([i/number_of_splits, 1 - i/number_of_splits])
        x_k = x_n[:, 0:k] # only up to k is used because we are trying to predict k+1
        numerator_terms.append(i / number_of_splits * S_k(b_i, x_k))
        denominator_terms.append(S_k(b_i, x_k))
    
    weighted_num = np.sum(np.array(numerator_terms))
    all_portfolios_denom = np.sum(np.array(denominator_terms))
    
    return np.array([weighted_num/all_portfolios_denom, 1 - weighted_num/all_portfolios_denom])

In [ ]:
# # Test Case

# temp_b = [np.array([1,0]), np.array([0,1]), np.array([0,1])]
# temp_x = x_n[:, 0:3]
# S_n(temp_b, temp_x, n=1)

# # Test

# b_next(x_n=x_n, k=5030)

In [ ]:
def optimal_rebalancing_strategy(df_stocks_relatives, 
                                 stocks, 
                                 number_of_splits=50, 
                                 b_init=np.array([0.50, 0.50])):
    
    x_1 = np.array(df_stocks_relatives[stocks[0]])
    x_2 = np.array(df_stocks_relatives[stocks[1]])
    x_n = np.vstack((x_1, x_2))

    # Simulate the whole trajectory now. We start with b_0 and obtain consecutive non-anticipating rebalancing portfolios

    all_b = []
    all_b.append(b_init)

    # Auxillary Variables

    total_runs = x_n.shape[1]
    universal_wealth_multiple = []

    for k in range(1, total_runs+1):

        if k % 500 == 0:
            print("{} Runs Remaining".format(total_runs-k))

        b_inc = b_next(x_n, k, number_of_splits=number_of_splits)
        all_b.append(b_inc)
        universal_wealth_multiple.append(S_n(all_b, x_n, n=k))  
    
    return universal_wealth_multiple, all_b

In [ ]:
def postprocess_output_for_two_stocks(df_stocks_relatives, stocks, df_stocks, 
                                      universal_wealth_multiple, all_b):

    # Universal Wealth Multiple DataFrame
    
    x_1 = np.array(df_stocks_relatives[stocks[0]])
    x_2 = np.array(df_stocks_relatives[stocks[1]])
    universal_mult = np.array(universal_wealth_multiple)
    stock_1 = [np.prod(x_1[0:i]) for i in range(1, x_1.shape[0]+1)]
    stock_2 = [np.prod(x_2[0:i]) for i in range(1, x_2.shape[0]+1)]
    
    df_wealth_multiplier = pd.DataFrame(data={"universal_portfolio": universal_mult,
                                             "stock_1_only": stock_1, 
                                             "stock_2_only": stock_2
                                             })
    
    df_wealth_multiplier["Date"] = df_stocks_relatives["Date"]

    # Portfolio Weights
    
    frac_stock_1 = [b[0] for b in all_b]
    frac_stock_2 = [b[1] for b in all_b]
    
    df_portfolio_weight = pd.DataFrame(data={stocks[0]: frac_stock_1, 
                                            stocks[1]: frac_stock_2})
    
    df_portfolio_weight["Date"] = df_stocks_relatives["Date"]
    
    # Minor refurbishing of date column
    
    df_wealth_multiplier.at[0, "Date"] = df_stocks["Date"][0]
    df_portfolio_weight.at[0, "Date"] = df_stocks["Date"][0]
    
    return df_wealth_multiplier, df_portfolio_weight

In [ ]:
def plot_wealth_multiplier(df_wealth_multiplier, stocks, 
                           title_text="Wealth Multiplier", yaxis_text="Multiplier"):

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df_wealth_multiplier["Date"],
                             y=df_wealth_multiplier["universal_portfolio"],
                             mode='lines',
                             name="Universal Portfolio", 
                             marker=dict(color='Black')))

    fig.add_trace(go.Scatter(x=df_wealth_multiplier["Date"],
                             y=df_wealth_multiplier["stock_1_only"],
                             mode='lines',
                             name=stocks[0], 
                             marker=dict(color='Red')))

    fig.add_trace(go.Scatter(x=df_wealth_multiplier["Date"],
                             y=df_wealth_multiplier["stock_2_only"],
                             mode='lines',
                             name=stocks[1], 
                             marker=dict(color='Blue')))

    fig.update_layout(title=title_text, yaxis_title=yaxis_text)
    
    return fig

In [ ]:
def plot_portfolio_allocation(df_portfolio_weight, stocks, 
                           title_text="Portfolio Weight", yaxis_text="Fraction of Portfolio (%) in "):

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df_portfolio_weight["Date"],
                             y=df_portfolio_weight[stocks[0]],
                             mode='lines',
                             name=stocks[0], 
                             marker=dict(color='Red')))

    fig.update_layout(title=title_text, yaxis_title= yaxis_text + stocks[0])
    
    return fig

## MAIN CODE

In [171]:
def main(stocks):

    # Load stocks dataframe

    # stocks = ["AMZN", "AAPL"]

    # Load Stock Data

    df_stocks = pd.read_csv("./data/stock_prices_8.csv")
    df_stocks_relatives = compute_price_relatives(df_stocks)


    prelim_stock = plot_stock_performance(df_stocks, stocks, 
                                          title_text="Stock Price", 
                                          yaxis_text="Stock Price ($)")

    prelim_multiplier = plot_stock_performance(df_stocks_relatives, stocks, 
                           title_text="Wealth Multiplier", 
                           yaxis_text="Daily Wealth Multiple")

    # Optimal Constantly Rebalanced

    df_hindsight, optimal_alloc, best_multiplier = hindsight_constantly_rebalanced_portfolio(
        df_stocks_relatives, 
        stocks)

    hindsight_optimal = plot_optimal_hindsight_rebalanced_portfolio(df_hindsight, stocks)

    # Universal Portfolios

    universal_wealth_multiple, all_b = optimal_rebalancing_strategy(df_stocks_relatives, stocks)

    df_wealth_multiplier, df_portfolio_weight = postprocess_output_for_two_stocks(
        df_stocks_relatives, stocks, df_stocks, universal_wealth_multiple, all_b)

    universal_multiplier = plot_wealth_multiplier(df_wealth_multiplier, stocks)
    universal_allocation = plot_portfolio_allocation(df_portfolio_weight, stocks)

    return prelim_stock, prelim_multiplier, hindsight_optimal, universal_multiplier, universal_allocation

In [173]:
main(stocks=["AMZN", "AAPL"])

4531 Runs Remaining
4031 Runs Remaining
3531 Runs Remaining
3031 Runs Remaining
2531 Runs Remaining
2031 Runs Remaining
1531 Runs Remaining
1031 Runs Remaining
531 Runs Remaining
31 Runs Remaining


(Figure({
     'data': [{'mode': 'lines',
               'name': 'AMZN',
               'type': 'scatter',
               'x': array(['2000-07-31', '2000-08-01', '2000-08-02', ..., '2020-07-28',
                           '2020-07-29', '2020-07-30'], dtype=object),
               'y': array([  30.125   ,   30.25    ,   30.8125  , ..., 3000.330078, 3033.530029,
                           3051.879883])},
              {'mode': 'lines',
               'name': 'AAPL',
               'type': 'scatter',
               'x': array(['2000-07-31', '2000-08-01', '2000-08-02', ..., '2020-07-28',
                           '2020-07-29', '2020-07-30'], dtype=object),
               'y': array([  3.142018,   3.049264,   2.921729, ..., 373.01001 , 380.160004,
                           384.76001 ])}],
     'layout': {'template': '...', 'title': {'text': 'Stock Price'}, 'yaxis': {'title': {'text': 'Stock Price ($)'}}}
 }), Figure({
     'data': [{'mode': 'lines',
               'name': 'AMZN',
        